# Ejercicio 04: Evaluación de un Sistema de Recuperación de Información

El objetivo de este ejercicio es evaluar la efectividad de un sistema de recuperación de información utilizando métricas como *precisión*, *recall*, *F1-score*, *Mean Average Precision (MAP)* y *Normalized Discounted Cumulative Gain (nDCG)*.

Seguirás los siguientes pasos:

Descripción del Ejercicio

1. Proporcionar un Conjunto de Datos:
    * Corpus de Documentos: Utiliza el corpus del ejercicio anterior o un nuevo conjunto de documentos.
    * Consultas: Define un conjunto de consultas específicas.
    * Juicios de Relevancia: Proporciona una lista de qué documentos son relevantes para cada consulta.

2. Calcular Resultados de Búsqueda:
    * Obten los resultados ordenados de dos sistemas de recuperación para cada consulta.

3. Calcular las Métricas de Evaluación:
    * Calcular las siguientes métricas para cada sistema y consulta:
        * Precisión en el top-k (Prec@k)
        * Recall
        * F1-score
        * Mean Average Precision (MAP)
        * nDCG

4. Análisis y Comparación:
    * Comparar los resultados de los dos sistemas utilizando las métricas calculadas.
    * Discutir cuál sistema es más efectivo y por qué.

In [144]:
# Importar las librerías necesarias
import xml.etree.ElementTree as ET

queries = {
    1: "Impacto de la salud mental en el rendimiento académico de los estudiantes universitarios",
    2: "Actividades extracurriculares y bienestar emocional en el campus universitario",
    3: "Estrategias universitarias para reducir el estrés en estudiantes"
}
# Función para procesar el texto y extraer palabras clave
def process_text(text):
    # Convertir a minúsculas
    text = text.lower()
    # Reemplazar caracteres no alfanuméricos por espacios
    import re
    text = re.sub(r'[^a-záéíóúñü]+', ' ', text)
    # Tokenizar y eliminar palabras vacías si es necesario
    tokens = text.strip().split()
    return set(tokens)

# Paso 1: Leer y parsear el archivo XML
def parse_corpus(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    corpus = {}
    for doc in root.findall('document'):
        doc_id = int(doc.get('id'))
        title = doc.find('title').text
        keywords = doc.find('keywords').text
        author = doc.find('author').text
        date = doc.find('date').text
        # Procesar las palabras clave
        keyword_set = process_text(keywords)
        corpus[doc_id] = {
            'title': title,
            'keywords': keyword_set,
            'author': author,
            'date': date
        }
    return corpus

In [145]:
corpus = parse_corpus('/content/03ranking_corpus.xml')
print(corpus)

{1: {'title': 'El aumento de la telemedicina para el tratamiento de condiciones de salud crónicas.', 'keywords': {'tecnología', 'crónica', 'tratamiento', 'telemedicina', 'médica', 'salud'}, 'author': 'Dr. Juan Pérez', 'date': '2023-01-15'}, 2: {'title': 'Cómo la nutrición balanceada afecta el rendimiento académico y la salud mental en estudiantes.', 'keywords': {'mental', 'estudiantes', 'nutrición', 'académico', 'salud', 'rendimiento'}, 'author': 'Dra. María López', 'date': '2023-02-10'}, 3: {'title': 'Estudio sobre cómo las relaciones de amistad contribuyen al bienestar de los estudiantes en el campus.', 'keywords': {'campus', 'estudiantil', 'sociales', 'relaciones', 'bienestar', 'amistad'}, 'author': 'Miguel Rodríguez', 'date': '2023-03-05'}, 4: {'title': 'El rol de las bibliotecas universitarias en el fomento de la investigación académica.', 'keywords': {'academia', 'investigación', 'recursos', 'universitarias', 'bibliotecas'}, 'author': 'Lucía Martínez', 'date': '2023-04-20'}, 5: {

In [146]:
docs = []
for doc_id in corpus:
    docs.append(corpus[doc_id]['title'])

print(len(docs))
print(docs)

30
['El aumento de la telemedicina para el tratamiento de condiciones de salud crónicas.', 'Cómo la nutrición balanceada afecta el rendimiento académico y la salud mental en estudiantes.', 'Estudio sobre cómo las relaciones de amistad contribuyen al bienestar de los estudiantes en el campus.', 'El rol de las bibliotecas universitarias en el fomento de la investigación académica.', 'Cómo los espacios verdes en los campus universitarios pueden mejorar la concentración y reducir el estrés.', 'La influencia de la cultura universitaria en los hábitos saludables y el bienestar de los estudiantes.', 'La importancia del sueño en la salud mental y el rendimiento académico en jóvenes universitarios.', 'La influencia del apoyo familiar y social en el bienestar emocional de los estudiantes universitarios.', 'El impacto de los hábitos de lectura en el desarrollo cognitivo en jóvenes y adultos.', 'Avances en la tecnología para la medicina preventiva y su aplicación en jóvenes.', 'Impacto de la práct

In [147]:
vocab = set()
for doc_id in corpus:
    vocab.update(corpus[doc_id]['keywords'])
vocab = list(vocab)
print(len(vocab))
print(vocab)

115
['tecnologías', 'estudiantil', 'estudio', 'del', 'remotas', 'cultura', 'eventos', 'calidad', 'académica', 'innovación', 'relaciones', 'equilibrio', 'retención', 'concentración', 'tendencias', 'programas', 'comparación', 'impacto', 'recursos', 'becas', 'colaboración', 'sostenibilidad', 'en', 'artísticas', 'médica', 'verdes', 'salud', 'desarrollo', 'transporte', 'investigación', 'creatividad', 'talleres', 'agotamiento', 'psicológico', 'medicina', 'social', 'estrategias', 'ambiental', 'interacción', 'laboratorios', 'mindfulness', 'compartidas', 'campus', 'bicicletas', 'academia', 'servicios', 'trabajan', 'actividades', 'universidades', 'competencias', 'saludables', 'movilidad', 'sueño', 'sostenible', 'lectura', 'tecnológicos', 'telemedicina', 'universitaria', 'carga', 'actividad', 'tecnológicas', 'desempeño', 'comunidades', 'universidad', 'prevención', 'universitarios', 'edificios', 'estrés', 'educación', 'diseño', 'bienestar', 'académico', 'rendimiento', 'ejercicio', 'tecnología', 'e

# Calculo de la matriz tf-idf

In [148]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

#Funcion para obtener la matriz tf-idf usando sklearn

def tfidf_transform(vocab, matrix):
  count_vectorizer = CountVectorizer(vocabulary=vocab)
  tf_matrix = count_vectorizer.fit_transform(matrix)
  tfidf_transformer = TfidfTransformer()
  tfidf_matrix = tfidf_transformer.fit_transform(tf_matrix).toarray()
  return tfidf_matrix

In [149]:
#Obtener la matriz tf-idf de los docuemntos del corpus
tfidf_matrix = tfidf_transform(vocab, docs)
print(tfidf_matrix)

[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.38969591 ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]


In [150]:
#Obtener la matriz tf-idf de las queries
queries_matrix = []
for i, val in queries.items():
  queries_matrix.append(queries[i])

qr_tfidf_matrix = tfidf_transform(vocab, queries_matrix)
print(qr_tfidf_matrix)

[[0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.30251368
  0.         0.         0.         0.         0.17866945 0.
  0.         0.         0.30251368 0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.30251368
  0.         0.         0.         0.         0.         0.30251368
  0.30251368 0.         0.         0.         0.         0.
  0.60502736 0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0. 

# Calculo de la similitud coseno

In [151]:
#Calculo de la similitud coseno
from math import sqrt
def similitudCoseno(v1, v2):
    similitud = []

    for i in range(0, len(v1)):
        producto = 0
        sum1 = 0
        sum2 = 0
        for j in range(0, len(v1[0])):
            producto += v1[i][j] * v2[j]
            sum1 += v1[i][j] ** 2
            sum2 +=  v2[j] ** 2

        sim = producto/(sqrt(sum1) * sqrt(sum2))
        similitud.append(round(sim,3))
    return similitud

In [152]:
cosenoMatrix = []

for i in range(0,len(qr_tfidf_matrix)):
  cosenoMatrix.append(similitudCoseno(tfidf_matrix, qr_tfidf_matrix[i]))

print(cosenoMatrix)

[[0.436, 0.558, 0.269, 0.227, 0.111, 0.263, 0.541, 0.254, 0.291, 0.049, 0.531, 0.259, 0.388, 0.428, 0.233, 0.273, 0.118, 0.141, 0.333, 0.332, 0.302, 0.123, 0.477, 0.186, 0.025, 0.324, 0.232, 0.102, 0.401, 0.022], [0.0, 0.039, 0.297, 0.035, 0.175, 0.15, 0.067, 0.292, 0.056, 0.065, 0.062, 0.062, 0.167, 0.035, 0.595, 0.029, 0.17, 0.13, 0.032, 0.056, 0.031, 0.034, 0.159, 0.037, 0.312, 0.03, 0.153, 0.125, 0.032, 0.029], [0.0, 0.142, 0.125, 0.293, 0.237, 0.122, 0.085, 0.123, 0.071, 0.083, 0.333, 0.153, 0.289, 0.044, 0.078, 0.037, 0.041, 0.106, 0.306, 0.138, 0.04, 0.043, 0.09, 0.348, 0.042, 0.11, 0.108, 0.206, 0.041, 0.037]]


# Calculo de la similitud Jaccard

In [153]:
# Calculo de la similitud Jaccard

def similitudJaccard(v1, v2):
    jaccardMatrix = []
    for i in range(len(v1)):
        similitudMatrix = []
        for j in range(len(v2)):
            intersection = 0
            union = 0
            for k in range(len(v1[0])):
                if v1[i][k] and v2[j][k]:
                    intersection += 1
                if v1[i][k] or v2[j][k]:
                    union += 1
            similitud = intersection/union
            similitudMatrix.append(round(similitud, 3))
        jaccardMatrix.append(similitudMatrix)
    return jaccardMatrix

In [154]:
jaccardMatrix = similitudJaccard(qr_tfidf_matrix,tfidf_matrix)
print(len(jaccardMatrix))

3


# Ranking por similitud Coseno

In [155]:
rankingCoseno = {}  # Creamos un diccionario vacío para almacenar el ranking por consulta

for i in range(0, len(cosenoMatrix)):
    indices = {}  # Diccionario temporal para almacenar los índices de los documentos y sus similitudes
    for j in range(0, len(cosenoMatrix[i])):
        indiceqr = j  # Índice del documento
        indices[indiceqr] = cosenoMatrix[i][j]  # Asignamos la similitud
    rankingCoseno[i + 1] = dict(sorted(indices.items(), key=lambda item: item[1], reverse=True))  # Asignamos el ranking para la consulta

# Verificar si ranking es un diccionario
print(rankingCoseno)  # Esto devolverá True si ranking es un diccionario


{1: {1: 0.558, 6: 0.541, 10: 0.531, 22: 0.477, 0: 0.436, 13: 0.428, 28: 0.401, 12: 0.388, 18: 0.333, 19: 0.332, 25: 0.324, 20: 0.302, 8: 0.291, 15: 0.273, 2: 0.269, 5: 0.263, 11: 0.259, 7: 0.254, 14: 0.233, 26: 0.232, 3: 0.227, 23: 0.186, 17: 0.141, 21: 0.123, 16: 0.118, 4: 0.111, 27: 0.102, 9: 0.049, 24: 0.025, 29: 0.022}, 2: {14: 0.595, 24: 0.312, 2: 0.297, 7: 0.292, 4: 0.175, 16: 0.17, 12: 0.167, 22: 0.159, 26: 0.153, 5: 0.15, 17: 0.13, 27: 0.125, 6: 0.067, 9: 0.065, 10: 0.062, 11: 0.062, 8: 0.056, 19: 0.056, 1: 0.039, 23: 0.037, 3: 0.035, 13: 0.035, 21: 0.034, 18: 0.032, 28: 0.032, 20: 0.031, 25: 0.03, 15: 0.029, 29: 0.029, 0: 0.0}, 3: {23: 0.348, 10: 0.333, 18: 0.306, 3: 0.293, 12: 0.289, 4: 0.237, 27: 0.206, 11: 0.153, 1: 0.142, 19: 0.138, 2: 0.125, 7: 0.123, 5: 0.122, 25: 0.11, 26: 0.108, 17: 0.106, 22: 0.09, 6: 0.085, 9: 0.083, 14: 0.078, 8: 0.071, 13: 0.044, 21: 0.043, 24: 0.042, 16: 0.041, 28: 0.041, 20: 0.04, 15: 0.037, 29: 0.037, 0: 0.0}}


# Ranking por Similitud Jaccard

In [156]:
rankingJaccard = {}  # Creamos un diccionario vacío para almacenar el ranking por consulta

for i in range(0, len(jaccardMatrix)):
    indices = {}  # Diccionario temporal para almacenar los índices de los documentos y sus similitudes
    for j in range(0, len(jaccardMatrix[i])):
        indiceqr = j  # Índice del documento
        indices[indiceqr] = jaccardMatrix[i][j]  # Asignamos la similitud
    rankingJaccard[i + 1] = dict(sorted(indices.items(), key=lambda item: item[1], reverse=True))  # Asignamos el ranking para la consulta

# Verificar si ranking es un diccionario
print(rankingJaccard)  # Esto devolverá True si ranking es un diccionario


{1: {1: 0.6, 6: 0.5, 10: 0.462, 13: 0.417, 18: 0.385, 12: 0.375, 22: 0.333, 28: 0.308, 11: 0.286, 7: 0.267, 19: 0.267, 17: 0.25, 2: 0.214, 5: 0.214, 20: 0.214, 23: 0.214, 25: 0.214, 8: 0.2, 26: 0.2, 0: 0.182, 3: 0.154, 4: 0.143, 15: 0.143, 16: 0.143, 21: 0.143, 14: 0.133, 9: 0.071, 27: 0.071, 24: 0.067, 29: 0.067}, 2: {14: 0.364, 24: 0.273, 2: 0.25, 7: 0.214, 17: 0.182, 4: 0.167, 16: 0.167, 5: 0.154, 26: 0.143, 12: 0.111, 3: 0.083, 9: 0.083, 27: 0.083, 1: 0.077, 15: 0.077, 21: 0.077, 22: 0.077, 29: 0.077, 13: 0.071, 20: 0.071, 23: 0.071, 25: 0.071, 28: 0.071, 6: 0.067, 8: 0.067, 11: 0.067, 18: 0.067, 10: 0.062, 19: 0.062, 0: 0.0}, 3: {23: 0.3, 18: 0.273, 10: 0.25, 3: 0.222, 17: 0.222, 1: 0.2, 4: 0.2, 12: 0.2, 2: 0.182, 5: 0.182, 25: 0.182, 11: 0.167, 26: 0.167, 7: 0.154, 19: 0.154, 9: 0.1, 27: 0.1, 15: 0.091, 16: 0.091, 21: 0.091, 22: 0.091, 24: 0.091, 29: 0.091, 13: 0.083, 14: 0.083, 20: 0.083, 28: 0.083, 6: 0.077, 8: 0.077, 0: 0.0}}


# Calculo Métricas de Evaluación

In [161]:
juiciosRelevancia = {
    1: {0: 0, 1: 0, 2: 0, 3: 0, 4: 1, 5: 0, 6: 1, 7: 1, 8: 0,
        9: 0, 10: 1, 11: 1, 12: 1, 13: 1, 14: 0, 15: 0, 16: 0,
        17: 0, 18: 1, 19: 1, 20: 0, 21: 0, 22: 1, 23: 0, 24: 0,
        25: 0, 26: 0, 27: 0, 28: 0, 29: 0},

    2: {0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0,
        9: 0, 10: 0, 11: 0, 12: 1, 13: 0, 14: 1, 15: 0, 16: 0,
        17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0,
        25: 0, 26: 0, 27: 0, 28: 0, 29: 0},

    3: {0: 0, 1: 0, 2: 0, 3: 0, 4: 1, 5: 0, 6: 0, 7: 0, 8: 0,
        9: 0, 10: 1, 11: 1, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0,
        17: 0, 18: 0, 19: 1, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0,
        25: 0, 26: 0, 27: 0, 28: 0, 29: 0}
}

In [162]:
from math import log2

def precision_at_k(ranking, relevancias, k):
    relevant_count = sum([1 for doc in list(ranking.keys())[:k] if relevancias.get(doc, 0) == 1])
    return relevant_count / k

def recall(ranking, relevancias):
    relevant_retrieved = sum([1 for doc in ranking.keys() if relevancias.get(doc, 0) == 1])
    relevant_total = sum(relevancias.values())
    return relevant_retrieved / relevant_total if relevant_total > 0 else 0

def f1_score(precision, recall):
    return (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

def average_precision(ranking, relevancias):
    relevant_count = 0
    precision_sum = 0
    for i, doc in enumerate(ranking.keys(), start=1):
        if relevancias.get(doc, 0) == 1:
            relevant_count += 1
            precision_sum += relevant_count / i
    relevant_total = sum(relevancias.values())
    return precision_sum / relevant_total if relevant_total > 0 else 0

def mean_average_precision(ranking, relevancias):
    ap_sum = 0
    num_queries = len(ranking)
    for query, ranking in ranking.items():
        ap_sum += average_precision(ranking, relevancias.get(query, {}))
    return ap_sum / num_queries if num_queries > 0 else 0

def ndcg_at_k(ranking, relevancias, k):
    dcg = 0
    idcg = 0
    sorted_relevancias = sorted(relevancias.values(), reverse=True)
    for i, doc in enumerate(ranking.keys(), start=1):
        if i > k:
            break
        relevance = relevancias.get(doc, 0)
        dcg += relevance / log2(i + 1)
    for i, relevance in enumerate(sorted_relevancias, start=1):
        if i > k:
            break
        idcg += relevance / log2(i + 1)
    return dcg / idcg if idcg > 0 else 0


# Análisis y Comparación

In [172]:
from prettytable import PrettyTable

# Definir k
k = 5  # Puedes cambiar el valor de k según sea necesario

# Crear la tabla con encabezados
tabla = PrettyTable()
tabla.field_names = ["Sistema", "Consulta", f"Prec@{k}", "Recall", "F1-Score", "nDCG"]

# Calcular métricas para el sistema de Similitud Jaccard
for consulta, ranking in rankingJaccard.items():
    relevancias = juiciosRelevancia.get(consulta, {})
    prec_at_k = precision_at_k(ranking, relevancias, k)
    rec = recall(ranking, relevancias)
    f1 = f1_score(prec_at_k, rec)
    ndcg = ndcg_at_k(ranking, relevancias, k)
    # Agregar fila para el sistema Jaccard
    tabla.add_row(["Similitud Jaccard", consulta, round(prec_at_k, 3), round(rec, 3), round(f1, 3), round(ndcg, 3)])

# Calcular métricas para el sistema de Similitud Coseno
for consulta, ranking in rankingCoseno.items():
    relevancias = juiciosRelevancia.get(consulta, {})
    prec_at_k = precision_at_k(ranking, relevancias, k)
    rec = recall(ranking, relevancias)
    f1 = f1_score(prec_at_k, rec)
    ndcg = ndcg_at_k(ranking, relevancias, k)
    # Agregar fila para el sistema Coseno
    tabla.add_row(["Similitud Coseno", consulta, round(prec_at_k, 3), round(rec, 3), round(f1, 3), round(ndcg, 3)])

# Imprimir la tabla
print(tabla)


+-------------------+----------+--------+--------+----------+-------+
|      Sistema      | Consulta | Prec@5 | Recall | F1-Score |  nDCG |
+-------------------+----------+--------+--------+----------+-------+
| Similitud Jaccard |    1     |  0.8   |  1.0   |  0.889   | 0.661 |
| Similitud Jaccard |    2     |  0.2   |  1.0   |  0.333   | 0.613 |
| Similitud Jaccard |    3     |  0.2   |  1.0   |  0.333   | 0.195 |
|  Similitud Coseno |    1     |  0.6   |  1.0   |   0.75   |  0.53 |
|  Similitud Coseno |    2     |  0.2   |  1.0   |  0.333   | 0.613 |
|  Similitud Coseno |    3     |  0.2   |  1.0   |  0.333   | 0.246 |
+-------------------+----------+--------+--------+----------+-------+


El sistema de Similitud Jaccard parece ser más efectivo que el sistema de Similitud Coseno en general. En la mayoría de las métricas (especialmente en Prec@5, F1-score, y nDCG para la consulta 1), Jaccard obtiene mejores resultados